In [1]:
import json
import pandas as pd
from interlex_sql import interlex_sql 
from args_reader import read_args
from scicrunch_client_fast import scicrunch
from collections import defaultdict

In [2]:
args = read_args(keys='../keys.txt', production='-p')
sql = interlex_sql(engine_key=args['engine_key'])
sci = scicrunch(key='api_key', base_path=args['base_path'])

In [36]:
labels_to_ids = sql.get_labels_to_ids_dict()
#labels_to_ids = json.load(open('../dump/labels_to_ids_dict_complete_interlex_sql.json', 'r'))

In [42]:
for i, d in df.iterrows():
    if not labels_to_ids.get(' '.join(str(d.title).lower().strip().split())):
        print(str(d.title).lower().strip())

post‐injury surgical procedures
post‐injury conditions
post‐injury subject housing
bbb right  hindpaw subscore
bbb left  hindpaw subscore


In [41]:
for label in list(labels_to_ids):
    if 'injury surgical' in label:
        print(label)

preinjury surgical procedures
post-injury surgical procedures


In [44]:
labels_to_ilx = sql.get_labels_to_ilx_dict()

In [30]:
inputfile = '../../OCD-CDEs/ODC-CDEs.xlsx'
df = pd.read_excel(inputfile, sheetname='Overall rough MasterStruct')
#df = df.fillna()
#raw_data = json.load(open(inputfile, 'r')) 

In [6]:
df

,ODC_Domain,ODC_SubDomain,ODC_Assessment,title,ODC_VariableName,definition
0,INVESTIGATOR INFORMATION,INVESTIGATOR INFORMATION,NaN,Investigator,NaN,NaN
1,INVESTIGATOR INFORMATION,INVESTIGATOR INFORMATION,NaN,Laboratory,NaN,NaN
2,INVESTIGATOR INFORMATION,INVESTIGATOR INFORMATION,NaN,Institution or organization,NaN,NaN
3,EXPERIMENTAL INFORMATION,STUDY NAME,NaN,Baseline assessments and date (timing),NaN,NaN
4,EXPERIMENTAL INFORMATION,Animal Characteristics,NaN,Animal ID,AnimalID,Animal identification
5,EXPERIMENTAL INFORMATION,Animal Characteristics,NaN,Species,Species,Species of the animal being studied
6,EXPERIMENTAL INFORMATION,Animal Characteristics,NaN,Birth date,BirthDate,"Date (and time, if applicable and known) the s..."
7,EXPERIMENTAL INFORMATION,Animal Characteristics,NaN,Age value,AgeVal,Age at the time of injury
8,EXPERIMENTAL INFORMATION,Animal Characteristics,NaN,Age group,AgeGroup,"Value for subject's age, calculated as elapsed..."
9,EXPERIMENTAL INFORMATION,Animal Characteristics,NaN,Sex,SexSubjectGenotypTyp,"The difference between male and female, based ..."


In [95]:
for i, row in df.iterrows():
    label = row.title.lower().strip()
    if labels_to_ids.get(label):
        print(row.title,',',labels_to_ids.get(label),',',labels_to_ilx.get(label))
        #print(row)

Investigator , 5687 , ilx_0105686
Laboratory , 12818 , ilx_0112817
Animal ID , 12869 , ilx_0112868
Species , 10849 , ilx_0110848
Sex , 12867 , ilx_0112866
Pre-injury subject housing , 15039 , ilx_0115035
Preinjury conditions , 15038 , ilx_0115034
Preinjury surgical procedures , 15033 , ilx_0115029
Injury group assignment , 15040 , ilx_0115036
Survival time , 12875 , ilx_0112874


In [8]:
raw_data = df.to_dict('records')

In [7]:
col_info = {
    'ODC_Domain':15034,
    'ODC_SubDomain': 15035,
    'ODC_Assessment': 15036,
    'ODC_VariableName': 15037,#used to be heading 304378,
}
source = {'annotation_tid':15024,'value':'ODC-SCI'}

In [27]:
#see if any labels already exist
#create a format 
#definition, label, synonyms, superclass_label, superclass_ilx, superclass_tid

already_added_terms = []
output_data = defaultdict(dict)
for i, d in df.iterrows():    
    
    output_data[d.title].update({
        'label':d.title,
        'type':'cde',
        'annotations':[{'value':' '.join([w.lower().capitalize() for w in str(d[k]).split()]), 'annotation_tid':v} for k, v in col_info.items() if type(d[k])==str] + [source],
        })
    if type(d.definition) != float:
        if d.definition and len(d.definition) > 0:
            output_data[d.title].update({'definition':d.definition})
            
with open('/Users/tmsincomb/Desktop/OCD-CDEs/ocd-cdes-production-ready.json', 'w') as outfile:
    json.dump(output_data, outfile, indent=4)

In [33]:
output_data = defaultdict(dict)
for i, d in df.iterrows():
    if d.title == 'Institution or organization':
        output_data[d.title].update({
            'label':d.title,
            'definition':' ',
            'type':'cde',
            'annotations':[{'value':' '.join([w.lower().capitalize() for w in str(d[k]).split()]), 'annotation_tid':v} for k, v in col_info.items() if type(d[k])==str] + [source],
            })
        if type(d.definition) != float:
            if d.definition and len(d.definition) > 0:
                output_data[d.title].update({'definition':d.definition})
            
with open('/Users/tmsincomb/Desktop/OCD-CDEs/ocd-cdes-production-ready.json', 'w') as outfile:
    json.dump(output_data, outfile, indent=4)

In [35]:
sci.getAnnotationsVIAtid([304379])

[========================================================================] 100%


=== Getting Annotations ===


SystemExit: {'errormsg': 'Could not complete request', 'success': False} with status code [401]

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [21]:
a = {0:0, 1:1}
for i in a[:]:
    print(i)

TypeError: unhashable type: 'slice'

Help on class int in module builtins:

class int(object)
 |  int(x=0) -> integer
 |  int(x, base=10) -> integer
 |  
 |  Convert a number or string to an integer, or return 0 if no arguments
 |  are given.  If x is a number, return x.__int__().  For floating point
 |  numbers, this truncates towards zero.
 |  
 |  If x is not a number or if base is given, then x must be a string,
 |  bytes, or bytearray instance representing an integer literal in the
 |  given base.  The literal can be preceded by '+' or '-' and be surrounded
 |  by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
 |  Base 0 means to interpret the base from the string as an integer literal.
 |  >>> int('0b100', base=0)
 |  4
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __ceil__(...)
 |      Ceiling of